In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
torch.manual_seed(1015)
# define 'device' to upload tensor in gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 데이터 불러오기 & 수정(모델에 넣을 수 있게)

In [ ]:
all = pd.read_csv("/content/drive/MyDrive/dacon/daconcup/Data/all_preped_01.csv")
print(all.shape)
print(all.columns)
all_raw = all.copy()

(555, 10)
Index(['date', '사용자', '세션', '신규방문자', '페이지뷰', 'cnt_signin', 'cnt_login',
       'cnt_sub', 'total_participants', 'isTrain'],
      dtype='object')


In [ ]:
# scaling (train을 사용한 scaler)
train = all.loc[all['isTrain']==1, :] # 스케일링 전

mini = train.iloc[:,1:-1].min()
size = train.iloc[:,1:-1].max() - train.iloc[:,1:-1].min()

all.iloc[:,1:-1] = (all.iloc[:,1:-1] -  mini) / size
train = all.loc[all['isTrain']==1, [col for col in all.columns if col != 'isTrain']] # 스케일링 후

input_window = 30
output_window = 7

window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window, 8)) # (데이터 수, time sequence, input_dim)
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 4))  # (데이터 수, time sequence, output_dim)

for start in range(train.shape[0] - (input_window + output_window)):
    end = start + input_window    
    window_x[start,:, :] = train.iloc[start : end                , 1: ].values
    window_y[start,:, :] = train.iloc[end   : end + output_window, 1:5].values

print(window_x.shape, window_y.shape)

(457, 30, 8) (457, 7, 4)


## 모델 생성 & 학습 

- pytorch 
    - [Encoder-Decoder Model for Multistep Time Series Forecasting Using PyTorch](https://towardsdatascience.com/encoder-decoder-model-for-multistep-time-series-forecasting-using-pytorch-5d54c6af6e60)
    - [seq2seq 모델 PyTorch로 구현하기 번역 및 정리](https://codlingual.tistory.com/91)

- keras
    - [Multi-Step LSTM Time Series Forecasting Models for Power Usage](https://machinelearningmastery.com/how-to-develop-lstm-models-for-multi-step-time-series-forecasting-of-household-power-consumption/)


In [ ]:
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
import tensorflow as tf

In [ ]:
# 모델 정의
model = Sequential()
model.add(LSTM(64, input_shape=(30, 8))) # (n_timesteps, n_features)
model.add(RepeatVector(7)) # n_outputs(7일)
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(4)))
model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer='adam')

# 모델 훈련
epochs = 500
model.fit(window_x, window_y, epochs=epochs, verbose=1)

Epoch 1/500
15/15 [==============================] - 4s 21ms/step - loss: 0.2024
Epoch 2/500
15/15 [==============================] - 0s 21ms/step - loss: 0.1032
Epoch 3/500
15/15 [==============================] - 0s 21ms/step - loss: 0.0957
Epoch 4/500
15/15 [==============================] - 0s 22ms/step - loss: 0.0810
Epoch 5/500
15/15 [==============================] - 0s 22ms/step - loss: 0.0801
Epoch 6/500
15/15 [==============================] - 0s 23ms/step - loss: 0.0746
Epoch 7/500
15/15 [==============================] - 0s 21ms/step - loss: 0.0759
Epoch 8/500
15/15 [==============================] - 0s 20ms/step - loss: 0.0704
Epoch 9/500
15/15 [==============================] - 0s 21ms/step - loss: 0.0716
Epoch 10/500
15/15 [==============================] - 0s 20ms/step - loss: 0.0712
Epoch 11/500
15/15 [==============================] - 0s 20ms/step - loss: 0.0734
Epoch 12/500
15/15 [==============================] - 0s 20ms/step - loss: 0.0691
Epoch 13/500
15/15 [=====

## 예측

In [ ]:
test = all.loc[all['isTrain']==0, [col for col in all.columns if col != 'isTrain']].dropna().reset_index(drop=True) # 한달만 예측(추가 데이터가 나와야 뒤의 한달을 예측할 수 있음)
submission = pd.read_csv("/content/drive/MyDrive/dacon/daconcup/Data/raw/submission.csv", encoding = 'euc-kr')


last_month = train.iloc[-30:,1:].values[np.newaxis,...] # <- 수정 후 array (30, 8) -> (1, 30, 8) using np.newaxis

for start in range((len(test) - output_window)//7 + 2):
    """ 
    i) train 뒤에서 30일을 뽑아서 test의 7일을 예측
    ii) 예측한 값을 train 뒤에 붙임
    위 과정 반복
    """
    start = start * 7
    next_week = model(last_month) # (1, 7, 4)

    if start/7 == (len(test) - output_window)//7 + 1: # 마지막 예측 부분(7일이 채 안됨)
        test.iloc[start : , 1:5] = next_week.numpy().reshape(-1, 4)[-test.iloc[start:, :].shape[0]:, :]
        next_week = test.iloc[start :, 1:].values[np.newaxis,...] # (1, x, 8) array
    else:
        test.iloc[start : start + output_window, 1:5] = next_week.numpy().reshape(output_window,4)
        next_week = test.iloc[start : start + 7, 1:].values[np.newaxis,...] # (1, 7, 8) array

    last_month = np.concatenate([last_month[:,7:,:], next_week], axis = 1)# <- 수정 후 (1, 23, 8) + (1, 7, 8) = (1, 30, 8)
    
    pred_week = next_week[0]
    pred_week = pred_week * size.values + mini.values
    pred_week = pred_week.astype(int)
    
    if start/7 == (len(test) - output_window)//7 + 1: # 마지막 예측 부분(7일이 채 안됨)
        submission.iloc[start : start + test.iloc[start :, 1:].shape[0], 1:] = pred_week[-test.iloc[start :, 1:].shape[0]:,:4] # 남는 날짜만큼만 추가
    else:
        submission.iloc[start : start + output_window, 1:] = pred_week[:, :4]
submission

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,3324,3407,811,76663
1,2020-11-10,3569,3561,893,81045
2,2020-11-11,3363,3362,780,74259
3,2020-11-12,3088,3079,628,66574
4,2020-11-13,2695,2713,502,56932
...,...,...,...,...,...
56,2021-01-04,0,0,0,0
57,2021-01-05,0,0,0,0
58,2021-01-06,0,0,0,0
59,2021-01-07,0,0,0,0


In [ ]:
submission_raw = submission.copy()
submission.columns = ['date', '사용자', '세션', '신규방문자', '페이지뷰']
df_all = pd.concat([all_raw[['date', '사용자', '세션', '신규방문자', '페이지뷰']].iloc[:-61, :], submission], axis=0).reset_index(drop=True)

In [ ]:
fig = make_subplots(shared_xaxes=True,rows=4, cols=1)
fig.add_trace(go.Scatter(x=df_all['date'], y=df_all['사용자'], name='users'), row=1, col=1)
fig.add_trace(go.Scatter(x=df_all['date'], y=df_all['세션'], name='sessions'), row=2, col=1)
fig.add_trace(go.Scatter(x=df_all['date'], y=df_all['신규방문자'], name='new_users'), row=3, col=1)
fig.add_trace(go.Scatter(x=df_all['date'], y=df_all['페이지뷰'], name='page views'), row=4, col=1)      

## 예측파일 저장 

In [ ]:
submission_raw.to_csv('/content/drive/MyDrive/dacon/daconcup/submission/submission+vars_seq2seq_lstm_hidden64_epoch500_mae.csv', index = False, encoding = 'euc-kr')